In [1]:
import pandas as pd
import psycopg2

In [23]:
# Устанавливаем соединение с базой данных
conn = psycopg2.connect(user="postgres", 
                        password="1234554321live##Test", 
                        host="localhost", 
                        database="postgres")

In [24]:
# Создаем объект курсора
cur = conn.cursor()

1) (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

2) (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

3) (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

4) (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

5) (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

6) (1 балл) Вывести всех клиентов, у которых нет транзакций.

7) (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

8) (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

1) (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [4]:
# Выполняем SQL-запрос
cur.execute("""
            select distinct(t.brand) as unique_brands 
            from homework_2.transaction as t 
            where standard_cost > 1500 and t.brand != 'NaN'
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
unique_brands = pd.DataFrame(results, columns=columns)
unique_brands

,unique_brands
0,OHM Cycles
1,Trek Bicycles
2,Solex
3,Giant Bicycles


2) (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [34]:
# Выполняем SQL-запрос
cur.execute("""
            select * from homework_2.transaction as t
            where t.transaction_date::date between '2017-04-01' and '2017-04-09' and t.order_status = 'Approved'
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
approved_transactions = pd.DataFrame(results, columns=columns)
approved_transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,17,79,2426,2017-04-03,false,Approved,Norco Bicycles,Standard,medium,medium,1555.58,818.01
1,19,54,2268,2017-04-06,true,Approved,WeareA2B,Standard,medium,medium,1292.84,13.44
2,23,37,2001,2017-04-08,true,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,83,0,3398,2017-04-01,true,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07
4,89,0,2682,2017-04-04,true,Approved,OHM Cycles,Road,high,large,12.01,7.21
...,...,...,...,...,...,...,...,...,...,...,...,...
526,19655,0,336,2017-04-09,true,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30
527,19853,7,3072,2017-04-02,false,Approved,Trek Bicycles,Road,low,medium,980.37,234.43
528,19899,57,325,2017-04-06,false,Approved,WeareA2B,Touring,medium,large,1890.39,260.14
529,19968,0,2751,2017-04-06,false,Approved,WeareA2B,Standard,medium,medium,60.34,45.26


3) (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [7]:
# Выполняем SQL-запрос
cur.execute("""
            select distinct(job_title) from homework_2.customer as c
            where c.job_title like 'Senior%' 
            and c.job_industry_category in ('IT', 'Financial Services')
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
seniors = pd.DataFrame(results, columns=columns)
seniors

,job_title
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


4) (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [16]:
# Выполняем SQL-запрос
cur.execute("""
            select distinct(t.brand)
            from homework_2.transaction as t 
            left join homework_2.customer c 
            on t.customer_id = c.customer_id
            where c.job_industry_category in ('Financial Services')
            and t.brand != 'NaN'
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
brands_fs = pd.DataFrame(results, columns=columns)
brands_fs

,brand
0,OHM Cycles
1,Trek Bicycles
2,WeareA2B
3,Solex
4,Norco Bicycles
5,Giant Bicycles


5) (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [25]:
# Выполняем SQL-запрос
cur.execute("""
            select 
            distinct c.customer_id, c.first_name, c.last_name
            from homework_2.transaction as t 
            left join homework_2.customer c 
            on t.customer_id = c.customer_id
            where t.online_order = 'true'
            and t.brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
            limit 10
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
brands_online = pd.DataFrame(results, columns=columns)
brands_online

,customer_id,first_name,last_name
0,742,Dexter,Robelin
1,3123,Tina,Riggulsford
2,20,Basile,Firth
3,1894,Patten,Laytham
4,3426,Ron,Dilon
5,1179,Kerry,Pashenkov
6,3491,Leanna,Cromb
7,2201,Trisha,Basset
8,1521,Pernell,Duffett
9,2431,Alvy,Tyndall


6) (1 балл) Вывести всех клиентов, у которых нет транзакций.

In [33]:
# Выполняем SQL-запрос
cur.execute("""
            select * from homework_2.customer as c 
            where c.customer_id not in 
            (select distinct t.customer_id from homework_2.transaction as t)
            order by c.customer_id
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
customers_without_orders = pd.DataFrame(results, columns=columns)
customers_without_orders

,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,852,Andie,Bonney,Female,2000-11-04,Compensation Analyst,Financial Services,Affluent Customer,N,Yes,94 Anhalt Way,3139,VIC,Australia,7
1,869,Addia,Abels,Female,1984-03-11,Account Representative I,Financial Services,High Net Worth,N,Yes,02377 Maywood Trail,2287,NSW,Australia,7
2,1373,Shaylynn,Epsley,Female,1958-09-23,Director of Sales,Financial Services,Mass Customer,N,Yes,0 Grasskamp Pass,3170,VIC,Australia,10
3,2074,Roslyn,Rawdall,Female,1997-06-11,NaN,Financial Services,Mass Customer,N,No,95483 Washington Junction,2505,NSW,Australia,9
4,2660,Hunt,Scollard,Male,1963-11-15,NaN,Retail,High Net Worth,N,Yes,359 Briar Crest Road,2155,NSW,Australia,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
503,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
504,3998,Sarene,Woolley,Unknown,1900-01-01,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
505,3999,Patrizius,NaN,Male,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


7) (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [36]:
# Выполняем SQL-запрос
cur.execute("""
            select * 
            from homework_2.customer as c
            where
            c.customer_id in (select distinct customer_id 
                              from homework_2.transaction
                              where 
                              standard_cost = (select max(standard_cost) from homework_2.transaction))
            and c.job_industry_category in ('IT')
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
clients_with_max_cost = pd.DataFrame(results, columns=columns)
clients_with_max_cost

,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,34,Jephthah,Bachmann,Unknown,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,90 Lawn Parkway,4805,QLD,Australia,4
1,168,Reggie,Broggetti,Unknown,1900-01-01,General Manager,IT,Affluent Customer,N,Yes,35386 Mockingbird Road,3749,VIC,Australia,4
2,255,Keeley,Kruger,Female,1954-01-15,Desktop Support Technician,IT,Mass Customer,N,No,8208 Parkside Point,2320,NSW,Australia,6
3,448,Robin,Crack,Female,1995-04-26,Software Engineer III,IT,High Net Worth,N,No,24919 Artisan Court,3015,Victoria,Australia,8
4,968,Wayne,Woodfin,Male,1955-06-16,Accountant IV,IT,High Net Worth,N,Yes,004 Lawn Trail,4227,QLD,Australia,5
5,975,Goldarina,Rzehorz,Unknown,1900-01-01,Automation Specialist IV,IT,Mass Customer,N,No,938 Monica Park,3173,VIC,Australia,9
6,1182,Carolann,Ottiwill,Female,1975-01-11,Data Coordiator,IT,Mass Customer,N,Yes,84 Westridge Hill,2061,NSW,Australia,10
7,1210,Shandie,Sprigg,Unknown,1900-01-01,Programmer II,IT,Mass Customer,N,No,275 Lindbergh Parkway,2086,NSW,Australia,11
8,1418,Valaria,Hugh,Female,1976-11-29,Senior Sales Associate,IT,Affluent Customer,N,Yes,0 Talisman Trail,2769,NSW,Australia,10
9,1672,Sharla,Creebo,Female,1963-04-27,Design Engineer,IT,Affluent Customer,N,Yes,1 Morning Circle,2250,NSW,Australia,8


8) (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [35]:
# Выполняем SQL-запрос
cur.execute("""
            select * 
            from homework_2.customer as c
            where
            c.customer_id in (select distinct customer_id 
                               from homework_2.transaction
                               where order_status = 'Approved'
                               and transaction_date between 
                               '2017-07-07' and '2017-07-17')
            and c.job_industry_category in ('IT', 'Health')
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# cur.execute("delete from sales.sales_tab where customer = 'Русский свет' and period = '2023-11-01'")
# Создаем датафрейм с нужными названиями столбцов
clients = pd.DataFrame(results, columns=columns)
clients

,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,22,Deeanne,Durtnell,Female,1962-12-10,NaN,IT,Mass Customer,N,No,52 Carey Alley,4740,QLD,Australia,5
1,28,Fee,Zellmer,Male,1973-09-30,Senior Quality Engineer,Health,Affluent Customer,N,Yes,2951 Petterle Place,2756,New South Wales,Australia,9
2,41,Basilius,Coupe,Male,1976-04-14,Food Chemist,Health,Mass Customer,N,No,2028 Lakewood Place,2480,New South Wales,Australia,3
3,47,Matthew,Jeaycock,Male,1992-10-22,Registered Nurse,Health,Affluent Customer,N,No,4853 Gulseth Avenue,4503,QLD,Australia,5
4,104,Odille,Panketh,Female,1978-06-06,Automation Specialist II,Health,Mass Customer,N,Yes,0410 Division Junction,2750,New South Wales,Australia,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,3255,Sutherlan,Truin,Unknown,1900-01-01,Engineer IV,IT,High Net Worth,N,No,12357 Arapahoe Parkway,4740,QLD,Australia,3
111,3288,Fair,Dewen,Unknown,1900-01-01,Engineer III,IT,High Net Worth,N,No,6 Golf Center,2042,NSW,Australia,10
112,3360,Joelie,Sherlaw,Female,1963-03-23,Quality Engineer,Health,Affluent Customer,N,No,62551 Del Mar Avenue,2300,NSW,Australia,6
113,3365,Karlens,Chaffyn,Unknown,1900-01-01,Engineer III,IT,Mass Customer,N,No,7234 Dawn Alley,4670,QLD,Australia,1
